```{contents}
```
## Concurrency Control

**Concurrency control** in LangGraph governs **how multiple nodes, agents, and executions run simultaneously** while preserving **correctness, consistency, and safety of shared state**.
It is the foundation for building **scalable, multi-agent, production-grade LLM systems**.

---

### **1. Why Concurrency Control is Required**

LLM workflows frequently involve:

* Parallel tool calls
* Multi-agent collaboration
* High-throughput request handling
* Long-running workflows with shared memory

Without proper control, systems suffer from:

| Failure             | Effect                  |
| ------------------- | ----------------------- |
| Race conditions     | Corrupted state         |
| Lost updates        | Incorrect results       |
| Deadlocks           | Stalled workflows       |
| Resource exhaustion | Cost explosion          |
| Non-determinism     | Irreproducible behavior |

---

### **2. Concurrency Model of LangGraph**

LangGraph uses a **cooperative, state-driven concurrency model** built on:

| Layer            | Responsibility       |
| ---------------- | -------------------- |
| Graph Scheduler  | Task ordering        |
| Async Runtime    | Concurrent execution |
| State Manager    | Safe state updates   |
| Reducer Engine   | Deterministic merges |
| Checkpoint Store | Recovery & replay    |

**Key property:**

> Nodes do not mutate shared state directly. They return **partial updates** that are merged deterministically.

---

### **3. State Consistency Mechanism**

#### **3.1 Partial State Updates**

Each node returns only the fields it modifies:

```python
def node_a(state):
    return {"counter": state["counter"] + 1}
```

#### **3.2 Reducers**

Reducers resolve concurrent updates safely:

```python
def merge_list(x, y): 
    return x + y
```

```python
builder.add_state_reducer("messages", merge_list)
```

This prevents **lost updates** and guarantees **eventual consistency**.

---

### **4. Parallel Execution**

LangGraph allows **fan-out / fan-in** concurrency:

```python
builder.add_edge("planner", "worker_a")
builder.add_edge("planner", "worker_b")
builder.add_edge("planner", "worker_c")
```

All workers execute **concurrently**, then merge state.

---

### **5. Conditional Concurrency Control**

Routing decisions determine concurrency dynamically:

```python
builder.add_conditional_edges(
    "router",
    route_fn,
    {"research": "research_agent", "compute": "math_agent"}
)
```

Only the selected branch executes.

---

### **6. Async & Task Scheduling**

Nodes may be **async functions**:

```python
async def tool_call(state):
    result = await api.fetch(state["query"])
    return {"data": result}
```

LangGraph’s scheduler manages:

* Awaiting async tasks
* Coordinating parallel execution
* Enforcing execution order

---

### **7. Failure Safety in Concurrent Execution**

| Mechanism          | Purpose                         |
| ------------------ | ------------------------------- |
| Checkpointing      | Resume after crash              |
| Idempotent updates | Prevent duplication             |
| Retry policies     | Recover from transient failures |
| Timeouts           | Prevent deadlocks               |
| Recursion limits   | Stop infinite loops             |

```python
graph.invoke(input, config={"recursion_limit": 20})
```

---

### **8. Multi-Agent Concurrency**

Each agent executes as an independent node with:

* Isolated local reasoning
* Shared global state
* Deterministic merge

```python
builder.add_node("agent_a", agent_a)
builder.add_node("agent_b", agent_b)
```

---

### **9. Production Guarantees**

| Guarantee       | How LangGraph Provides It |
| --------------- | ------------------------- |
| Consistency     | Reducers + checkpoints    |
| Scalability     | Async runtime + fan-out   |
| Fault tolerance | Replay + resume           |
| Observability   | Tracing & logging         |
| Determinism     | State merge discipline    |

---

### **10. Mental Model**

LangGraph concurrency behaves like a **transactional workflow engine**:

> **Parallel computation → deterministic merge → consistent state**

This model enables **safe, scalable, autonomous AI systems**.



### Demonstration

In [2]:
# =========================
# LangGraph Concurrency Demo
# =========================

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import asyncio
import random
import operator

# -------------------------
# 1. Define Shared State with Reducers
# -------------------------

class State(TypedDict):
    logs: Annotated[list[str], operator.add]
    total: Annotated[int, operator.add]

# -------------------------
# 2. Concurrent Worker Nodes
# -------------------------

async def worker_async(name, value):
    await asyncio.sleep(random.random())  # simulate real async work
    return {"logs": [f"{name} finished"], "total": value}

async def worker_a(state):
    return await worker_async("A", 10)

async def worker_b(state):
    return await worker_async("B", 20)

async def worker_c(state):
    return await worker_async("C", 30)

# -------------------------
# 3. Start & Final Nodes
# -------------------------

def start(state):
    return state

def summarize(state):
    return {"logs": [f"Final total = {state['total']}"]}

# -------------------------
# 4. Build Graph
# -------------------------

builder = StateGraph(State)

builder.add_node("start", start)
builder.add_node("A", worker_a)
builder.add_node("B", worker_b)
builder.add_node("C", worker_c)
builder.add_node("summary", summarize)

builder.set_entry_point("start")

# Fan-out: start branches to all workers
builder.add_edge("start", "A")
builder.add_edge("start", "B")
builder.add_edge("start", "C")

# Fan-in: all workers converge to summary
builder.add_edge("A", "summary")
builder.add_edge("B", "summary")
builder.add_edge("C", "summary")

builder.add_edge("summary", END)

graph = builder.compile()

# -------------------------
# 5. Execute
# -------------------------

async def main():
    result = await graph.ainvoke({"logs": [], "total": 0})
    
    print("Logs:")
    for line in result["logs"]:
        print(" •", line)
    
    print("\nTotal:", result["total"])

await main()


Logs:
 • A finished
 • B finished
 • C finished
 • Final total = 60

Total: 60
